# CSC 593

## Week 11

In [ ]:
#setup.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Commonly-used widget functions
from ipywidgets import interact, interact_manual, interactive_output

import ipywidgets as widgets

# To explicitly display widgets.
from IPython.display import display

%matplotlib notebook

### If that first cell gives you an error:

Run this in a Terminal (Mac) or Anaconda Powershell Prompt (Windows):

`conda install -c conda-forge voila`

### Course Evaluations:

https://uri.campuslabs.com/courseeval/ce/csc/593/0001


### Interactivity with IPyWidgets

IPyWidgets allows you to add interactive elements to your Jupyter notebooks. 

The documentation is [here](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html). There's also a useful tutorial [on Github](https://github.com/jupyter-widgets/tutorial). You can just `git clone` it to your computer.

For the most basic usage, just create a function with arguments that have default values, then call the IPyWidgets `interact()` function:

In [ ]:
#Basic usage
def f(x=5):
    return x + 10

interact(f);

`interact()` takes a function, then a value for each argument of that function (here, it just needs `x`). Since we're giving it a single value for `x`, it will use that as the default. Since the default is an integer, it gives us a slider that returns an integer with a range of (x-10) to (x+10).

When the value of the slider changes, `f()` is automatically called with the new value for `x`, and the output is printed below the slider.

You can also define the default value in the function declaration:

In [ ]:
def g(x=5):
    return x+20

interact(g, x=(0, 10));

`interact()` uses `x`'s default value from the function declaration as its default. We can now set an explicit range for `x` on our slider.

It's also possible to define a "step" value for the slider. If we set it (or any other value for `x`) to a float, `interact()` will recognize this and provide a slider that can handle float values.

In [ ]:
#You can also specify a "step"
interact(g, x=(0, 10, .1));

#### Practice

Write a function that takes one argument and returns the square of its input. Set its argument's default to 3.


(Remember the exponent operator is `**`, or you can use the [`np.power()` function](https://docs.scipy.org/doc/numpy/reference/generated/numpy.power.html).)

In [ ]:
print(4**2)
print(np.power(4,2))

In [ ]:
#write your function here:


Now create an interactive slider that calls your function with an integer.

Create a new slider that gives your function a floating-point number.

### As a Decorator

We can also use `interact()` as a decorator, which simplifies our code in some cases.

In [ ]:
#Same effect using a decorator
@interact(x=(0, 10, .01))
def h(x=7.5):
    return x+100

### Type Detection

`interact()` recognizes the arguments' data types and automatically chooses an appropriate widget.

In [ ]:
#interact() will choose the appropriate widget for many data types.
@interact()
def i(x=True):
    return 10 if x else 0

In [ ]:
@interact()
def j(x="WHATEVER"):
    return x.lower()

#### Practice

The `reverse()` function, takes a string as input and returns the string backwards. Create an interactive text box that runs `reverse()` on its input. 

In [ ]:
#From the IPyWidgets tutorial, section 02.01.
#https://github.com/jupyter-widgets/tutorial
def reverse(x):
    return x[::-1]

reverse('This is backwards text.')

Create a check box that writes 'yes' when checked, 'no' otherwise.

### Interactive visualizations

In [ ]:
#setup
iris = sns.load_dataset('iris')

sns.set()
plt.rc('figure', figsize=[10,5])

Remember that the function we give to `interact()` can do anything we want. This provides a convenient way to create visualizations that can be manipulated interactively.

In [ ]:
def k(x=25):
    sns.stripplot('petal_width', 'petal_length', data=iris[:x], jitter=True, color='red', alpha=.3)
    plt.plot()

In [ ]:
interact(k, x=(0, iris.count()[0]));

One issue with interactive plotting is lag--it takes time to draw the graph, especially as we drag the slider and the chart is redrawn repeatedly. One fix is to use `interact_manual()`, which adds a button to trigger the function.

In [ ]:
#Plotting can be slow. 
interact_manual(k, x=(0, iris.count()[0]));

#### Practice

You can use the [`np.random.normal()`](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.normal.html) function to generate random numbers on a normal distribution. Create a widget that draws a histogram with between 100 and 1000 normally distributed random numbers with a mean of 10 and a standard deviation of 3.

(You can draw a histogram with `plt.hist()` or `sns.distplot()`)

### Multiple arguments

We can use multiple arguments the same as single ones.

In [ ]:
def l(a=.25, jit=True):
    sns.stripplot('petal_width', 'petal_length', data=iris, jitter=jit, color='red', alpha=a)
    plt.plot()

interact(l, a=(0,1,.05), lag=False);

### Specifying widget types

If you don't like `interact()`'s default widget for something, you can specify a different one. There's a list at https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html.

This also allows you to configure things like the labels on buttons and placement.

In [ ]:
interact(l, 
         a=widgets.BoundedFloatText(value=.25, min=0, max=1, step=.05, description="Opacity"),
         jit=widgets.ToggleButton(value=True, description="Jitter?"));

If you don't want to be able to change an argument, use the `fixed` widget.

In [ ]:
interact(l, a=widgets.fixed(.3), lag=False);

### `continuous_update`

An alternative to `interact_manual()` is the `continuous_update` option on slider widgets. This graph won't change when you're manipulating the slider, only when you let go.

In [ ]:
#Another option for dealing with slow functions.
interact(k, x=widgets.IntSlider(25, min=1, max=iris.count()[0], continuous_update=False));

Here's another, more complex example, showing a line plot.

In [ ]:
#Staging.
#from the National Conference of State Legislatures.
unemp = pd.read_html('http://www.ncsl.org/research/labor-and-employment/state-unemployment-update.aspx')[0].iloc[:,:11]

#Reshape the table into "Long" format.
unemp = pd.melt(unemp, id_vars='State', var_name="Month", value_name="Rate")
unemp.head()

In [ ]:
def plotUnemp(states=[]):
    plt.figure(figsize=(12,8))
    data = unemp[unemp.State.isin(states)]
    sns.lineplot('Month', 'Rate', 'State', data=data)

In [ ]:
interact(plotUnemp, states = widgets.SelectMultiple(
                                    options=unemp.State,
                                    value=('Connecticut', 'Maine', 'Massachusetts', 
                                           'New Hampshire', 'Rhode Island', 'Vermont'),
                                    description='State'
))

### Practice

Update your normal histogram plotting function. Add a True/False option to change the type of plot:

* If you're using `plt.hist()`, set the `histtype` argument to `'bar'` if False, `'step'` if True.
* If you're using `sns.distplot()`, set the `kde` option equal to your new argument.

Create an interaction using both the count and type options to update the histogram.

### Placement

This works, but it's not particularly attractive:

In [ ]:
def m(x=150, jit=True, species='all'):
    plt.figure(figsize=(12,6))
    dat = iris if species=='all' else iris[iris.species==species]
        
    sns.stripplot('petal_width', 'petal_length', hue='species',
                  data=dat[:x], jitter=jit, alpha=.3)
    plt.show()

In [ ]:
interact(m, 
         x=widgets.BoundedIntText(150, min=1, max=iris.count()[0], description='Count', continuous_update=False),
         jit=widgets.ToggleButton(value=True, description="Jitter"), 
         species = widgets.Dropdown(options=np.append(iris.species.unique(), 'all')
                                    , value='all', description='Species')
);

There are a number of [layout widgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Container/Layout-widgets) available to help you control the placement of your widgets.

In [ ]:
x = widgets.IntSlider(150, min=1, max=iris.count()[0], continuous_update=False)
jit = widgets.ToggleButton(value=True, description="Jitter")
species = widgets.ToggleButtons(options=np.append(iris.species.unique(), 'all'), value='all', 
                        description='Species')

# An HBox lays out its children horizontally
ui = widgets.HBox([x, jit, species])

out = interactive_output(m, {'x': x, 'jit': jit, 'species': species})

display(ui, out)

Here's the same interaction with the options to the side of the graph instead of above. 

Also note that when we create widgets this way, the visible instances of them are synchronized (when you change the graph below, the one above also changes.

In [ ]:
ui = widgets.VBox([x, jit, species])
display(widgets.HBox([ui, out]))

#### Practice

Add arguments to your normal histogram function for mean and standard deviation (the first two arguments to `np.random.normal()`. Create an interaction with controls for all four arguments above or beside the histogram.